In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json
import math

import sys
sys.path.insert(0, '../ToolBox')

import ZScore

import os, sys
import cPickle

In [2]:
train = pd.read_csv('./train.csv')
keys = ["OutcomeType", "AnimalType", "SexuponOutcome", "AgeuponOutcome", "Breed", "Color"]

In [3]:
dict_temp = {}

dict_temp["OutcomeType"] = np.array([val for val in train.OutcomeType.unique() if str(val) != 'nan'and str(val).lower() != 'unknown'])
dict_temp["OutcomeType"] = np.sort(dict_temp["OutcomeType"]) #To make easier the construction of submission.

In [4]:
dict_temp["Breed"] = np.array([val for val in train.Breed.unique() if str(val) != 'nan'and  'unknown' not in str(val).lower()])
np.set_printoptions(threshold='nan')

In [5]:
dict_temp["AnimalType"] = np.array([val for val in train.AnimalType.unique() if str(val) != 'nan' and 'unknown' not in str(val).lower()])
np.set_printoptions(threshold='nan')

In [6]:
dict_temp["SexuponOutcome"] = np.array([val for val in train.SexuponOutcome.unique() if str(val) != 'nan' and 'unknown' not in str(val).lower()])

In [7]:
dict_temp["AgeuponOutcome"] = train.AgeuponOutcome.unique()

In [8]:
dict_temp["Color"] = np.array([val for val in train.Color.unique() if str(val) != 'nan' and str(val).lower() != 'unknown'])

dict_colors = {}

i = 1

for value in dict_temp["Color"]:
    if value not in dict_colors and '/' not in value:
        dict_colors[value] = i
        i+=1
    elif value not in dict_colors and  '/' in value:
        temp_val = value.split('/')
        temp = '{1}/{0}'.format(temp_val[0], temp_val[1]) 
        if temp not in dict_colors:
            dict_colors[value] = i
            i += 1

def getColorCode(colorValue):
    if colorValue in dict_colors:
        return dict_colors[colorValue]
    splits = colorValue.split('/')
    if len(splits) > 1: #Must be 2
        val = '{1}/{0}'.format(splits[0], splits[1])
        if val in dict_colors:
            return dict_colors[val]
    return 0

In [9]:
def convertAgeToNum(age):
    if str(age).isdigit():
        return age
    elif str(age) == 'nan':
        return 0
    else:
        age = age.strip()
        age = age.lower()
        splits = age.split()
        if "year" in splits[1]:
            return float(splits[0]) * 365
        elif "month" in splits[1]:
            return float(splits[0]) * 30
        elif "week" in splits[1]:
            return float(splits[0]) * 7
        elif "day" in splits[1]:
            return float(splits[0])

In [10]:
train_transformed = []
for idx, row in train.iterrows():
    lis = []
    for key in keys:
        if key == "AgeuponOutcome":
            age_in_days = convertAgeToNum(row[key])
            lis.append(age_in_days)
        elif key == 'Color':
            if str(row[key]) != "nan" and "unknown" not in str(row[key]).lower():
                lis.append(getColorCode(row[key]))
            else:
                lis.append(0)
        elif str(row[key]) != "nan" and "unknown" not in str(row[key]).lower():
            temp = np.where(dict_temp[key] == row[key])[0][0]+1
            lis.append(temp)
        else:
            lis.append(0)
    train_transformed.append(np.array(lis))

In [11]:
train_transformed = np.array(train_transformed, np.int16)

In [12]:
labels = [x[0] for x in train_transformed]
train = [x[1:] for x in train_transformed]

In [13]:
test = pd.read_csv('./test.csv')
test_keys = ["AnimalType", "SexuponOutcome", "AgeuponOutcome", "Breed", "Color"]

In [14]:
test_transformed = []
for idx, row in test.iterrows():
    lis = []
    for key in test_keys:
        if key == "AgeuponOutcome":
            age_in_days = convertAgeToNum(row[key])
            lis.append(age_in_days)
        elif key == 'Color':
            if str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
                lis.append(getColorCode(row[key]))
            else:
                lis.append(0)
        elif str(row[key]) != "nan" and str(row[key]).lower() != "unknown":
            temp = np.where(dict_temp[key] == row[key])
            if len(temp[0]) > 0:
                lis.append(temp[0][0]+1)
            else:
                lis.append(0)
        else:
            lis.append(0)
    test_transformed.append(np.array(lis))

In [15]:
train_transformed = np.array(train, np.float32)
test_transformed = np.array(test_transformed, np.float32)

In [16]:
dict_zscore = ZScore.init()
train_mean, train_stddev = dict_zscore["z_score_compute"](train_transformed)
normalized_train_data = dict_zscore["z_score_normalize"](train_transformed, train_mean, train_stddev)
normalized_test_data = dict_zscore["z_score_normalize"](test_transformed, train_mean, train_stddev)

In [17]:
if 'pre_processed_data' not in os.listdir("."):
    os.mkdir('pre_processed_data')
labels = np.array(labels)
labels = np.reshape(labels, [26729, 1])
train_final = np.append(np.array(labels), normalized_train_data, axis=1)

In [18]:
f = open('pre_processed_data/train_preprocessed.pkl', 'w')
cPickle.dump(np.array(train_final, np.float32), f, -1)
f.close()

g = open('pre_processed_data/test_preprocessed.pkl', 'w')
cPickle.dump(np.array(normalized_test_data, np.float32), g, -1)
g.close()